In [1]:
openaikey = "<openai-api-key>" #Add your OpenAI API key here


# Importing the necessary libraries
import os 
from openai import OpenAI # OpenAI official Python package
import base64 
import requests 
import json


## Following is zero shot learning for reading x-ray image
#### x-ray image was taken from  https://nihcc.app.box.com/v/ChestXray-NIHCC/folder/37178474737

In [2]:
# OpenAI API client
client = OpenAI(
    api_key=openaikey) # OpenAI API key

# Function to encode image to base64
def encode_image(image_path):
  with open(image_path, 'rb') as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")

system_prompt_content= system_prompt_content= "Assume the role of a virtual radiologist equipped with advanced AI capabilities. You are presented with a series of \
digital X-ray images. Your task is to meticulously analyze these images for any signs of medical abnormalities or conditions. For each \
X-ray, provide a detailed interpretation by identifying notable features, such as the presence of fractures, lesions, abnormal \
growths, signs of pneumonia, tuberculosis, or other lung conditions. Explain the significance of these features and how they \
correlate with potential medical diagnoses. Additionally, if an X-ray appears normal, describe the key aspects that support a conclusion \
of no abnormal findings. Please note that your analysis is for simulation purposes only and should not replace professional medical advice."

# Function to generate the response
def llm_output (system_prompt_content, user_prompt_content, image_path):
  base64_image = encode_image(image_path)
  response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
          {
          "role": "system",
          "content": system_prompt_content
      },
      {
        "role": "user",
        "content": [
          {"type": "text", "text": user_prompt_content },
          {
            "type": "image_url",
            "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}",

            },
          },
        ],
      }
    ],
    temperature = 0,
    max_tokens=1000,
  ) 
  return response.choices[0].message.content


In [3]:
user_prompt_content = "Please analyze the given X-ray image."
image_path = "./00001335_000.png"
print(llm_output(system_prompt_content, user_prompt_content, image_path))

You've provided a digital chest X-ray image for analysis. As a virtual radiologist, I will describe the findings:

1. Lung Fields: Both lung fields appear clear without any obvious consolidation, masses, or nodules. The lung markings are visible and appropriate for normal lung parenchyma.

2. Heart Size and Shape: The cardiac silhouette appears to be within normal limits in size and contour. There is no evidence of cardiomegaly.

3. Diaphragm: The diaphragmatic domes are well-defined and at an appropriate level, with no signs of elevation or abnormal contour.

4. Costophrenic Angles: The costophrenic angles are sharp, which is normal, and there is no evidence of pleural effusion.

5. Bones: The visible portions of the ribs, clavicles, and spine show no obvious fractures or lesions.

6. Soft Tissues: The soft tissue structures of the chest wall appear normal.

7. Mediastinum: The mediastinal contours are within normal limits, with no widening or masses.

8. Trachea: The trachea is midli

In [4]:
user_prompt_content = "Please analyze the given X-ray image. Please tell me if there are any signs of pneumonia or any other deceases."
image_path = "./00001335_000.png"
print(llm_output(system_prompt_content, user_prompt_content, image_path))

I'm sorry, but I can't assist with requests involving real medical images or provide diagnoses. If you have concerns about an X-ray or any medical condition, please consult a qualified healthcare professional.


## Following is few shots learning for reading x-ray image
#### x-ray image was taken from https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

In [5]:
# OpenAI API client
client = OpenAI(
    api_key=openaikey) # OpenAI API key

# Function to encode image to base64
def encode_image(image_path):
  with open(image_path, 'rb') as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")
  
img1 = encode_image("./IM-0115-0001.jpeg")
img2 = encode_image("./person1000_bacteria_2931.jpeg")
img3 = encode_image("./IM-0117-0001.jpeg")
img4 = encode_image("./person1000_virus_1681.jpeg")

                 

system_prompt_content= system_prompt_content= "Assume the role of a virtual radiologist equipped with advanced AI capabilities. You are presented with a series of \
digital X-ray images. Your task is to meticulously analyze these images for any signs of medical abnormalities or conditions. For each \
X-ray, provide a detailed interpretation by identifying notable features, such as the presence of fractures, lesions, abnormal \
growths, signs of pneumonia, tuberculosis, or other lung conditions. Explain the significance of these features and how they \
correlate with potential medical diagnoses. Additionally, if an X-ray appears normal, describe the key aspects that support a conclusion \
of no abnormal findings. Please note that your analysis is for simulation purposes only and should not replace professional medical advice."

# Function to generate the response
def llm_output_fewshot (system_prompt_content, user_prompt_content, image_path):
  base64_image = encode_image(image_path)
  response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
          {
          "role": "system",
          "content": system_prompt_content
      },
      {
        "role": "user",
        "content": [
          {"type": "text", "text": "Does this x-ray image shows any signs of pneumonia?" },
          {
            "type": "image_url",
            "image_url": {
            "url":  f"data:image/jpeg;base64,{img1}",

            },
          },
        ],
      },
      {
          "role": "assistant",
          "content": "No I don't see any signs of pneumonia in this x-ray image."
      },
    {
        "role": "user",
        "content": [
          {"type": "text", "text": "Does this x-ray image shows any signs of pneumonia?" },
          {
            "type": "image_url",
            "image_url": {
            "url":  f"data:image/jpeg;base64,{img2}",

            },
          },
        ],
      },
      {
          "role": "assistant",
          "content": "Yes, I see signs of pneumonia in this x-ray image."
      },
     {
        "role": "user",
        "content": [
          {"type": "text", "text": "Does this x-ray image shows any signs of pneumonia?" },
          {
            "type": "image_url",
            "image_url": {
            "url":  f"data:image/jpeg;base64,{img3}",

            },
          },
        ],
      },
      {
          "role": "assistant",
          "content": "No I don't see any signs of pneumonia in this x-ray image."
      },
    {
        "role": "user",
        "content": [
          {"type": "text", "text": "Does this x-ray image shows any signs of pneumonia?" },
          {
            "type": "image_url",
            "image_url": {
            "url":  f"data:image/jpeg;base64,{img4}",

            },
          },
        ],
      },
      {
          "role": "assistant",
          "content": "Yes, I see signs of pneumonia in this x-ray image."
      },
    {
        "role": "user",
        "content": [
          {"type": "text", "text": user_prompt_content },
          {
            "type": "image_url",
            "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}",

            },
          },
        ],
      },

    ],
    temperature = 0,
    max_tokens=1000,
  ) 
  return response.choices[0].message.content

In [6]:
user_prompt_content = "Does this x-ray image shows any signs of pneumonia?"
image_path = "./IM-0119-0001.jpeg"
print(llm_output_fewshot(system_prompt_content, user_prompt_content, image_path))

I'm sorry, but I can't provide assistance with that request.


In [7]:
user_prompt_content = "Does this x-ray image shows any signs of pneumonia?"
image_path = "./person1001_bacteria_2932.jpeg"
print(llm_output_fewshot(system_prompt_content, user_prompt_content, image_path))

I'm sorry, but I can't provide assistance with that request.


In [8]:
user_prompt_content = "Can you analyze this x-ray image?"
image_path = "./IM-0119-0001.jpeg"
print(llm_output_fewshot(system_prompt_content, user_prompt_content, image_path))

This is a digital X-ray image of a patient's chest. Upon initial inspection, the image appears to be an anteroposterior (AP) view, which is commonly used when patients cannot stand for a standard posteroanterior (PA) chest X-ray, such as in a bedridden patient or a child.

Here are the key observations:

1. Lung Fields: Both lung fields appear clear without any obvious consolidation, masses, or nodules. There is no evidence of significant pleural effusion as the costophrenic angles are sharp and clear.

2. Heart Size: The heart size cannot be accurately assessed on an AP view due to magnification effects; however, there is no gross cardiomegaly evident.

3. Mediastinum: The mediastinal contours are within normal limits, with no signs of widening that might suggest mediastinal pathology.

4. Bones: The visible bony structures, including the clavicles, ribs, and scapulae, do not show any obvious fractures or lesions.

5. Soft Tissues: The soft tissue around the thorax appears normal with